In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import torch
import matplotlib.pyplot as plt
import os
from joblib import Parallel, delayed
from tqdm import tqdm
from multiprocessing import Pool, cpu_count

In [2]:

def load_rnafm(filename, seq_len, L_max):
    """
    Load data from a .npy file and convert it to an N x N matrix.

    Parameters:
    - filename: Path to the .npy file.
    - N: Dimension of the square matrix.

    Returns:
    - bpp_matrix: N x N matrix reconstructed from the input file.
    """
    # Load the structured array from the .npy file
    data = np.load(filename)

    # Create an empty N x N matrix
    bpp_matrix = np.zeros((seq_len, seq_len))

    # Fill the matrix with the probabilities from the loaded data
    bpp_matrix[data["pos_1"], data["pos_2"]] = data["probabilities"]

    bpp_matrix = bpp_matrix + bpp_matrix.T - np.diag(np.diag(bpp_matrix))
    full = np.zeros((L_max, L_max))
    full[:seq_len, :seq_len] = bpp_matrix
    return torch.tensor(full)


def generate_base_pair_matrixv1(file_path, L):
    """
    Reads a TXT file of base pair probabilities and generates an n x n matrix.

    Args:
    - file_path (str): Path to the TXT file.

    Returns:
    - np.array: An n x n matrix of base pair probabilities.
    """
    # Read the data using pandas
    data = pd.read_csv(file_path, sep=" ", header=None, names=["pos1", "pos2", "prob"])

    # Find the largest position in the 'pos1' column
    largest_position = data["pos1"].max()

    ids = torch.from_numpy(data[["pos1", "pos2"]].values.astype(int))
    matrix = torch.zeros((L, L))
    matrix[ids[:, 0] - 1, ids[:, 1] - 1] = torch.from_numpy(data["prob"].values).float()
    matrix[ids[:, 1] - 1, ids[:, 0] - 1] = torch.from_numpy(data["prob"].values).float()

    return matrix


def extra_bpp_from_numpy(filename, N, seq_len=None):
    """
    Load data from a .npy file and convert it to an N x N matrix.

    Parameters:
    - filename: Path to the .npy file.
    - N: Dimension of the square matrix.

    Returns:
    - bpp_matrix: N x N matrix reconstructed from the input file.
    """
    # Load the structured array from the .npy file
    if filename.parent.stem in ["rnafm", "rnaformerv1"]:
        full = load_rnafm(filename, seq_len, N)
    else:
        data = np.load(filename)
        # Create an empty N x N matrix
        bpp_matrix = np.zeros((N, N))
        # Fill the matrix with the probabilities from the loaded data
        bpp_matrix[data["pos_1"], data["pos_2"]] = data["probabilities"]
        full = torch.tensor(bpp_matrix)

    return full

def dot_to_adjacencyv0(dot_notation, n):
    adjacency_matrix = np.zeros((n, n), dtype=int)
    stack = []
    for i, char in enumerate(dot_notation):
        if char == "(":
            stack.append(i)
        elif char == ")":
            j = stack.pop()
            adjacency_matrix[i][j] = adjacency_matrix[j][i] = 1

    return adjacency_matrix

In [3]:
class CFG:
    pathss = Path("../eda/rmdb_data.v1.3.0_ss.parquet")
    split_id = Path('../eda/fold_split.csv')
    
df = pd.read_parquet('rmdb_data.v1.3.0_ss.parquet')
df['L'] = df['sequence'].apply(len)




In [4]:
df

,sequence_id,sequence,ss_full,ss_full_mfe,L
0,b2c4f7dfcbeb,GGGAAACUGCCUGAUGGAGGGGGAUAACUACUGGAAACGGUAGCUA...,(((..((...(((((((.(.((..((.((((((....)))))).))...,-35.500000,110
1,c917ebd9ebb1,CGGAAACUGCCUGAUGGAGGGGGAUAACUACUGGAAACGGUAGCUA...,.((..((...(((((((.(.((..((.((((((....)))))).))...,-33.599998,110
2,85d63477c1f0,GCGAAACUGCCUGAUGGAGGGGGAUAACUACUGGAAACGGUAGCUA...,(((....)))(((((((.(.((..((.((((((....)))))).))...,-32.900002,110
3,362b98907e64,GGCAAACUGCCUGAUGGAGGGGGAUAACUACUGGAAACGGUAGCUA...,((((......(((((((.(.((..((.((((((....)))))).))...,-36.400002,110
4,9735bca2a802,GGGUAACUGCCUGAUGGAGGGGGAUAACUACUGGAAACGGUAGCUA...,(((((....((.(((((.(.((..((.((((((....)))))).))...,-37.599998,110
...,...,...,...,...,...
66818,49ab4913e71d,AAUAAGAGAGUGUAUCUAGGGUUCCGGUCAAUAGAUGUCUGGUCCG...,.....((((.((((((...((..((((.((.....)).)))).))....,-53.000000,179
66819,086ef0f2e5ec,AAUAAGAGAGUGUAUCUAGGGUUCCGGUCAAUAGAUGUCUGGUCCG...,.....((((.((((((...((..((((.((.....)).)))).))....,-53.000000,180
66820,2750b76343ee,AAUAAGAGAGUGUAUCUAGGGUUCCGGUCAAUAGAUGUCUGGUCCG...,......(((.....))).(..(((((((..(((..((((((........,-54.099998,181
66821,412de5b993a1,AAUAAGAGAGUGUAUCUAGGGUUCCGGUCAAUAGAUGUCUGGUCCG...,......(((.....))).(..(((((((..(((..((((((........,-54.099998,182


In [5]:
OUT = Path('bpp/rmdb_data/comb')
os.makedirs(OUT, exist_ok=True)


In [6]:
def convert(row):
    L = row.L
    bpp_fn = row.sequence_id  # You might need to convert this to Path object if it's not already.
    ss_full = row.ss_full

    extra_bpp = ["vienna_2", "contrafold_2", "rnaformerv1", 'eternafold', 'rnafm']
    extra_bpp_path = Path("bpp/rmdb_data")
    names = ["vienna_2", "contrafold_2", "rnaformerv1", 'eternafold', 'rnafm', 'ss_vienna']

    bpp_extra = [
        extra_bpp_from_numpy(extra_bpp_path / f"{i}/{bpp_fn}.npy", L, seq_len=L).numpy()
        for i in extra_bpp
    ]  + [dot_to_adjacencyv0(ss_full, L)]
    
    bpp_extra_d = {s : np.array(d).astype(np.float16) for s, d in zip(names, bpp_extra)}

    # Assuming OUT is a predefined path
    np.savez_compressed(OUT / f"{row.sequence_id}.npz", **bpp_extra_d)

# This function will be used by Pool.map which expects a single-argument function
def worker(index):
    row = df.iloc[index]
    convert(row)
    
with Pool(processes=16) as pool:
    max_ = df.shape[0]
    with tqdm(total=max_) as pbar:
        for _ in pool.imap_unordered(worker, range(max_)):
            pbar.update()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 66823/66823 [03:05<00:00, 360.15it/s]


In [ ]:
def get_original_dict(row):
    L = row.L
    bpp_fn = row.sequence_id  # You might need to convert this to Path object if it's not already.
    ss_full = row.ss_full

    extra_bpp = ["vienna_2", "contrafold_2", "rnaformerv1", 'eternafold', 'rnafm']
    extra_bpp_path = Path("bpp/rmdb_data")
    names = ["vienna_2", "contrafold_2", "rnaformerv1", 'eternafold', 'rnafm', 'ss_vienna']

    bpp_extra = [
        extra_bpp_from_numpy(extra_bpp_path / f"{i}/{bpp_fn}.npy", L, seq_len=L).numpy()
        for i in extra_bpp
    ]  + [dot_to_adjacencyv0(ss_full, L)]
    
    bpp_extra_d = {s : np.array(d).astype(np.float16) for s, d in zip(names, bpp_extra)}
    return bpp_extra_d

def get_saved(row):
    L = row.L
    bpp_fn = row.sequence_id  # You might need to convert this to Path object if it's not already.
    ss_full = row.ss_full

    extra_bpp = ["vienna_2", "contrafold_2", "rnaformerv", 'eternafold', 'rnafm', 'ss_vienna']
    OUT = Path('bpp/rmdb_data/comb')  # replace with your actual output path
    data = np.load(OUT / f"{row.sequence_id}.npz")
    return {i: data[i] for i in extra_bpp}

In [ ]:
data = get_original_dict(df.iloc[1491])

In [ ]:
df.query("L>300")

In [ ]:
for i in data.keys():
    print(i)
    plt.imshow(data[i])
    plt.pause(0.1)

In [ ]:
def compare(saved, original):
    assert (np.all([np.allclose(saved[k], original[k]) for k in saved.keys()]))
    
indexs = np.random.choice(np.arange(df.shape[0]), 100)
for index in tqdm(indexs):
    row = df.iloc[index]
    saved = get_saved(row)
    original = get_original_dict(row)
    compare(saved, original)

In [ ]:
np.random.choice